In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
training_set = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train.csv')
validation_set = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation.csv')
test_set = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test.csv')

In [3]:
# Import the os module to interact with the operating system, 
# which allows you to construct file paths and check if files exist.
import os

# Define a function that will be used to connect image paths to a pandas DataFrame for training set.
def connect_images_to_df_training_set(df):
    # Define the base path where all your images are stored.
    # This is the common path shared by all images.
    base_path = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train'

    # Create an empty list to store the paths of the images.
    images = []

    # Iterate over each row in the DataFrame.
    # 'iterrows()' returns both the index (idx) and the row content (row) for each row in the DataFrame.
    for idx, row in df.iterrows():
        # Extract the filename of the image from the 'imageFile' column of the DataFrame.
        image_name = row['imageFile']

        # Use 'os.path.join' to combine the base path with the image filename.
        # This creates the full path to the image file.
        image_path = os.path.join(base_path, image_name)

        # Check if the image file exists at the specified path.
        if os.path.exists(image_path):
            # If the file exists, append the path to the 'images' list.
            images.append(image_path)
        else:
            # If the file does not exist, append 'None' to indicate a missing file.
            # The commented out print statement is useful for debugging to see which files are missing.
            images.append(None)  # Image not found
            # print(f"Image not found for: {image_path}")  # Debug print

    # Add a new column 'image_path' to the DataFrame.
    # This column contains the full path to each image.
    df['image_path'] = images
    
# Define a function that will be used to connect image paths to a pandas DataFrame for validation set.
def connect_images_to_df_validation_set(df):
    # Define the base path where all your images are stored.
    # This is the common path shared by all images.
    base_path = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation/BTTAIxNYBG-validation'

    # Create an empty list to store the paths of the images.
    images = []

    # Iterate over each row in the DataFrame.
    # 'iterrows()' returns both the index (idx) and the row content (row) for each row in the DataFrame.
    for idx, row in df.iterrows():
        # Extract the filename of the image from the 'imageFile' column of the DataFrame.
        image_name = row['imageFile']

        # Use 'os.path.join' to combine the base path with the image filename.
        # This creates the full path to the image file.
        image_path = os.path.join(base_path, image_name)

        # Check if the image file exists at the specified path.
        if os.path.exists(image_path):
            # If the file exists, append the path to the 'images' list.
            images.append(image_path)
        else:
            # If the file does not exist, append 'None' to indicate a missing file.
            # The commented out print statement is useful for debugging to see which files are missing.
            images.append(None)  # Image not found
            # print(f"Image not found for: {image_path}")  # Debug print

    # Add a new column 'image_path' to the DataFrame.
    # This column contains the full path to each image.
    df['image_path'] = images

# Define a function that will be used to connect image paths to a pandas DataFrame for test set.
def connect_images_to_df_test_set(df):
    # Define the base path where all your images are stored.
    # This is the common path shared by all images.
    base_path = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test/BTTAIxNYBG-test'

    # Create an empty list to store the paths of the images.
    images = []

    # Iterate over each row in the DataFrame.
    # 'iterrows()' returns both the index (idx) and the row content (row) for each row in the DataFrame.
    for idx, row in df.iterrows():
        # Extract the filename of the image from the 'imageFile' column of the DataFrame.
        image_name = row['imageFile']

        # Use 'os.path.join' to combine the base path with the image filename.
        # This creates the full path to the image file.
        image_path = os.path.join(base_path, image_name)

        # Check if the image file exists at the specified path.
        if os.path.exists(image_path):
            # If the file exists, append the path to the 'images' list.
            images.append(image_path)
        else:
            # If the file does not exist, append 'None' to indicate a missing file.
            # The commented out print statement is useful for debugging to see which files are missing.
            images.append(None)  # Image not found
            # print(f"Image not found for: {image_path}")  # Debug print

    # Add a new column 'image_path' to the DataFrame.
    # This column contains the full path to each image.
    df['image_path'] = images

In [4]:
connect_images_to_df_training_set(training_set)
connect_images_to_df_validation_set(validation_set)

In [5]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import math

# Define the number of classes
num_classes = 10

# Define image dimensions - changing this makes this take much longer
image_height = 224 
image_width = 224

# Define batch size
batch_size = 32 # experiment with

# Define number of epochs
num_epochs = 50

steps_per_epoch = math.ceil(len(training_set) / batch_size)
validation_steps = math.ceil(len(validation_set) / batch_size)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.config.experimental.list_physical_devices('GPU'):
    print("TensorFlow will run on GPU")
else:
    print("TensorFlow will run on CPU by default")

2024-03-23 20:04:16.442129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 20:04:16.442227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 20:04:16.570770: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available:  2
TensorFlow will run on GPU


In [6]:
from sklearn import metrics
from tensorflow.keras.applications import ResNet101V2

#TensorFlow Stuff
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as lyrs
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [7]:
# Define ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, # only thing here was rescale / added brightness_range and channel_shift_range
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   zoom_range=0.2, 
                                   brightness_range=[0.8,1.2],  
                                   channel_shift_range=20)  
train_generator = train_datagen.flow_from_dataframe(
    dataframe=training_set,  # Use the training_set dataframe
    directory=None,  # No directory needed since full paths are provided in 'image_path'
    x_col='image_path',  # Use the 'image_path' column for image paths
    y_col='classLabel',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset = "training",
    seed = 42
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=validation_set,  # Use the validation_set dataframe
    directory=None,  # No directory needed since full paths are provided in 'image_path'
    x_col='image_path',  # Use the 'image_path' column for image paths
    y_col='classLabel',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=42,
)

Found 81946 validated image filenames belonging to 10 classes.
Found 10244 validated image filenames belonging to 10 classes.


In [8]:
#Import Resnet Model
model_base = ResNet101V2(
    input_shape=(224,224,3),
    include_top=False,
)
model_base.trainable = True

for layers in model_base.layers[:-30]:
    layers.trainable=False

#Transfer Learning Model
inputs=tf.keras.Input(shape=(224,224,3))
x=model_base(inputs)
x=lyrs.GlobalAveragePooling2D()(x)
x= lyrs.Dropout(0.5)(x)
outputs=lyrs.Dense(10, activation="softmax", 
                     kernel_regularizer=tf.keras.regularizers.l2(1e-3))(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet101v2 (Functional)        │ (None, 7, 7, 2048)     │    42,626,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,647,050 (162.69 MB)

 Trainable params: 14,464,010 (55.18 MB)

 Non-trainable params: 28,183,040 (107.51 MB)

In [ ]:
checkpointer = ModelCheckpoint('best_model.keras',
                               verbose=1, 
                               save_best_only= True, 
                               monitor='val_accuracy',  
                               mode='max'  # Ensure mode is set to 'max' since you're looking to maximize accuracy
                              )
early_stopping = EarlyStopping(monitor= 'val_accuracy', patience= 6)
# verbose is 3 instead of 6 because of the bug where every other epoch gives no accuracy or loss

model.compile(loss="categorical_crossentropy", 
              optimizer=Adam(learning_rate=1e-4), 
              metrics=["accuracy"])
model_history=model.fit(x=train_generator,  # Use train_generator here
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,  # Use validation_generator here
    validation_steps=validation_steps,  # Ensure you define validation_steps
    epochs=num_epochs,
    callbacks=[checkpointer, early_stopping])

Epoch 1/50
   1/2561 ━━━━━━━━━━━━━━━━━━━━ 21:55:59 31s/step - accuracy: 1.0000 - loss: 0.0155

W0000 00:00:1711241849.355228      85 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1094/2561 ━━━━━━━━━━━━━━━━━━━━ 18:25 753ms/step - accuracy: 0.9761 - loss: 0.0748

W0000 00:00:1711242672.890352      85 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2561/2561 ━━━━━━━━━━━━━━━━━━━━ 0s 751ms/step - accuracy: 0.9762 - loss: 0.0746

...


Epoch 1: val_accuracy improved from -inf to 0.97306, saving model to best_model.keras


W0000 00:00:1711243866.024047      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2561/2561 ━━━━━━━━━━━━━━━━━━━━ 2049s 788ms/step - accuracy: 0.9762 - loss: 0.0746 - val_accuracy: 0.9731 - val_loss: 0.1048
Epoch 2/50

Epoch 2: val_accuracy did not improve from 0.97306
2561/2561 ━━━━━━━━━━━━━━━━━━━━ 0s 29us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/50


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1124/2561 ━━━━━━━━━━━━━━━━━━━━ 16:27 687ms/step - accuracy: 0.9783 - loss: 0.0674

In [11]:
connect_images_to_df_test_set(test_set)

In [12]:
import numpy as np
from tensorflow.keras.models import load_model

# Define ImageDataGenerator for preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a test generator
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_set,
    directory=None,  
    x_col='image_path',  # Column in test_set that contains the image paths
    y_col=None,  # No labels
    target_size=(image_height, image_width),  # Same as training/validation
    batch_size=batch_size,  
    class_mode=None,  # Since we're predicting
    shuffle=False  # Important for keeping track of filenames
)

# Load the best model saved by the ModelCheckpoint
best_model = load_model('best_model.keras')

# Now use this best_model to make predictions
predictions = best_model.predict(test_generator, steps=int(np.ceil(len(test_set)/batch_size)))
predicted_classes = np.argmax(predictions, axis=1)  # Convert probabilities to class labels

# Create a DataFrame with 'uniqueID' and predicted 'classID'
submission_df = pd.DataFrame({
    'uniqueID': test_set['uniqueID'],  # Assuming 'uniqueID' is a column in test_set
    'classID': predicted_classes
})

# Ensure the DataFrame matches the format of the sample submission
submission_df = submission_df.reset_index(drop=True)

submission_df.head()

Found 30690 validated image filenames.


/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 690 variables whereas the saved optimizer has 64 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  2/960 ━━━━━━━━━━━━━━━━━━━━ 1:58 124ms/step

W0000 00:00:1711241100.656967      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


960/960 ━━━━━━━━━━━━━━━━━━━━ 393s 402ms/step


W0000 00:00:1711241486.427997      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


,uniqueID,classID
0,1,1
1,9,7
2,10,4
3,14,1
4,16,6


In [13]:
submission_df.to_csv('submission.csv', index=False)